In [1]:
###reference:
    ###https://towardsdatascience.com/machine-learning-with-the-titanic-dataset-7f6909e58280



def concat_df(train_data,test_data):
    return pd.concat([train_data,test_data],sort=True).reset_index(drop=True)

def divide_df(all_data):
    return all_data.loc[:890],all_data.loc[891:].drop(['Survived'],axis=1)

In [ ]:
import pandas as pd

train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

#df_all=concat_df(train_data,test_data)

###JHU not necessary code
###dfs=[train_data,test_data]

In [ ]:
print("train_data_contains: "+str(len(train_data))+" row and "+str(len(train_data.columns))+" columns")
print("test_data_contains: "+str(len(test_data))+" row and "+str(len(test_data.columns))+" columns")

In [ ]:
print("first_3_row_of_train_data")
display(train_data.head(3))
print("first_3_row_of_test_data")
display(test_data.head(3))

In [ ]:
print("missing_values_in_train_data")
display(train_data.isnull().sum())
print("missing_value_in_test_data")
display(test_data.isnull().sum())

In [ ]:
df_all=concat_df(train_data,test_data)

In [ ]:
####2 data cleansing Age

In [ ]:
print("missing_age_values_in_total_data_set: "+str(df_all['Age'].isnull().sum()))
print("missing_percentage: "+str(round(df_all['Age'].isnull().sum()/len(df_all)*100.0))+" %")

In [ ]:
###JHU
import seaborn as sb

train_data['Age'].plot.hist(alpha=0.4)

In [ ]:
###JHU
train_data.pivot(columns='Pclass').Age.plot(kind='hist',stacked=True,alpha=0.4)

In [ ]:
###JHU
train_data.groupby(['Pclass'])['Age'].plot.hist(alpha=0.4,legend=True)

In [ ]:
###JHU

sb.histplot(data=train_data,x="Age",hue="Pclass",multiple="stack",alpha=0.4)

In [ ]:
print('median_for_Age_seperated_by_Pclass:')
display(train_data.groupby('Pclass')['Age'].median())
print('median_for_Age_seperated_by_Pclass_and_Sex:')
display(train_data.groupby(['Pclass','Sex'])['Age'].median())
print('bumber_of_cases:')
display(train_data.groupby(['Pclass','Sex'])['Age'].count())

In [ ]:
#replace the missing value with the medians of each group
df_all['Age']=df_all.groupby(['Pclass','Sex'])['Age'].apply(lambda x:x.fillna(x.median()))
df_all

In [ ]:
####2 data cleansing Fare

In [ ]:
df_all.loc[df_all['Fare'].isnull()]

In [ ]:
#loc cases which are similiar to Mr.Thomas and use the median of fare to replace the missing for his data set
mr_thomas=df_all.loc[(df_all['Pclass']==3)&(df_all['SibSp']==0)&(df_all['Embarked']=='S')]['Fare'].median()
print(mr_thomas)
df_all.loc[df_all['Fare'].isnull(),'Fare']=mr_thomas

In [ ]:
####2 data cleansing Cabin

In [ ]:
display(train_data['Cabin'].unique())
print('there_are '+str(train_data['Cabin'].nunique())+' different_values_for_Cabin_and '+str(train_data['Cabin'].isnull().sum())+' cases_are_missing')

In [ ]:
#keep all first letters of Cabin in new variable and use "M" for each missing
df_all['Deck']=df_all['Cabin'].apply(lambda s:s[0] if pd.notnull(s) else 'M')

print(df_all['Deck'].sample(10))

import matplotlib.pyplot as pl

df_all[['Deck','Survived']].groupby('Deck')['Survived'].mean().plot(kind='bar')
pl.suptitle('Survival_rates_for_different_cabines')

In [ ]:
###JHU

df_all.groupby('Deck')['Survived'].mean().plot(kind='bar',alpha=0.4,legend=True)

In [ ]:
idx=df_all[df_all['Deck']=='T'].index
df_all.loc[idx,'Deck']='A'

###JHU i think the upper two lines are not necessary

df_all['Deck']=df_all['Deck'].replace(['A','B','C'],'ABC')
df_all['Deck']=df_all['Deck'].replace(['D','E'],'DE')
df_all['Deck']=df_all['Deck'].replace(['F','G'],'FG')

df_all['Deck'].value_counts()

In [ ]:
####2 data cleansing Embarked

In [ ]:
df_all.loc[df_all['Embarked'].isnull()]

In [ ]:
#check for passengers who were in passenger class 1, on deck abc and paid 80 or less for the tickets
df_all.loc[(df_all['Pclass']==1)&(df_all['Fare']<=80)&(df_all['Deck']=='ABC')]['Embarked'].value_counts()

In [ ]:
df_all.loc[df_all['Embarked'].isnull(),'Embarked']='S'

In [ ]:
####2 data cleansing Conclusion

In [ ]:
print("missing_in_all_data")
display(df_all.isnull().sum())

In [ ]:
print("missing_in_train_data")
display(train_data.isnull().sum())

###JHU why still keeps null Age values in train_data, since we have replaced
###median Age value by Pclass and Sex in df_all???

In [ ]:
####3 feature engineering Binning

In [ ]:
df_all.boxplot(column=['Fare'])###,figsize=(15,7))

In [ ]:
###JHU

sb.histplot(data=df_all,x="Fare",alpha=0.4)

In [ ]:
df_all.boxplot(column=['Age'])

In [ ]:
###JHU

sb.histplot(data=df_all,x="Age",alpha=0.4)

In [ ]:
###Benedikt Droste
###As you can see, there are outliers for both Age and Fare. 
###The range of values is much higher for Fare compared to Age.
###We will cut the istribution into pieces so that the outliers do not 
###irritate our algorithm. For Fare we will assign the same number of cases 
###to each category and for Age we will build the categories based on the
###values of the variable. This is also the difference between .cut and .qcut.
###With .cut, the bins are formed based on the values of the variable, 
###regardless of how many cases fall into a category. With .qcut we decompose 
###a distribution so that there are the same number of cases in each category.

###JHU ??? it could be the salling strategy which caused the Fare distribution
###JHU ??? they plaaned to sell ticket at all price range to cover the cost 
###JHU ??? but the market demand was not as they expected, so they sell low 
###JHU ??? priced ticket to young people in last minutes.

###df_all['Fare']=pd.qcut(df_all['Fare'],5)#### we must run this first to get
###the labels range then use labels parameter to distribute the bins

###JHU
df_all['Fare']=pd.qcut(df_all['Fare'],5,labels=["<7.8","<10.5","<21.5","<41.5","<512.3"])

###JHU
df_all['Age']=pd.cut(df_all['Age'],5,labels=["<16","<32","<48","<64","<80"])

print("For_Age,_each_category_has_a_different_number_of_cases:")
df_all['Age'].value_counts()

In [ ]:
print("For_Fare,_each_category_has_nearly_a_same_number_of_cases:")
df_all['Fare'].value_counts()

In [ ]:
###JHU
df_all[['Age','Survived']].sample(5)

In [ ]:
df_all[['Age','Survived']].groupby('Age')['Survived'].mean()

In [ ]:
df_all[['Fare','Survived']].groupby('Fare')['Survived'].mean()

In [ ]:
df_all[['Age','Survived']].groupby('Age')['Survived'].mean().plot(kind='bar',alpha=0.4)
pl.suptitle('Survival_rates_for_Age_categories')

In [ ]:
df_all[['Fare','Survived']].groupby('Fare')['Survived'].mean().plot(kind='bar',alpha=0.4)
pl.suptitle('Survivel_rate_for_Fare_categories')

In [ ]:
####3 Creat new features out of existing variables Family Size

###JHU why we do it this way

In [ ]:
df_all['Family_size']=df_all['SibSp']+df_all['Parch']+1
df_all['Family_size'].plot.hist()

In [ ]:
df_all['Family_size_bin']=df_all['Family_size'].map(lambda s:1 if s==1 else(2 if s==2 else(3 if 3<=s<=4 else(4 if s>=5 else 0))))

In [ ]:
df_all['Family_size_bin'].value_counts()

In [ ]:
df_all[['Family_size_bin','Survived']].groupby('Family_size_bin')['Survived'].mean().plot(kind='bar',alpha=0.4)

In [ ]:
####3 Creat new features out of existing variables Tickets


In [ ]:
df_all['Ticket_frequency']=df_all.groupby('Ticket')['Ticket'].transform('count')


###JHU .transform('count') and what why 'count'
###so 1 means most ticket sold category or something??

In [ ]:
df_all[['Ticket_frequency','Survived']].groupby('Ticket_frequency').mean()

In [ ]:
df_all[['Ticket_frequency','Survived']].groupby('Ticket_frequency')['Survived'].mean().plot(kind='bar',alpha=0.4)

In [ ]:
####3 Creat new features out of existing variables Title


In [ ]:
###JHU

df_all.sample(15)

In [ ]:
df_all['Title']=df_all['Name'].str.split(', ',expand=True)[1].str.split('.',expand=True)[0]
df_all['Is_married']=0
df_all['Is_married'].loc[df_all['Title']=='Mrs']=1


###JHU is that a joke or something? it seems many codes are reduandant or just
###JHU be there to help the reader learning to code in a proper way??

df_all['Title'].nunique()

In [ ]:
###JHU

df_all['Title'].unique()


In [ ]:
title_names=(df_all['Title'].value_counts()<10)

df_all['Title']=df_all['Title'].apply(lambda x:'Mice' if title_names.loc[x]==True else x)

df_all.groupby('Title')['Title'].count()

In [ ]:
###JHU
df_all[['Title','Survived']].groupby('Title')['Survived'].mean().plot(kind='bar',alpha=0.4)

In [ ]:
####3 Creat new features out of existing variables Survival Rates


In [ ]:
import string

def extract_surname(data):
    families=[]
    for i in range(len(data)):
        name=data.iloc[i]
        if '(' in name:
            name_no_bracket=name.split('(')[0]
        else:
            name_no_bracket=name
        family=name_no_bracket.split(',')[0]
        title=name_no_bracket.split(',')[1].strip().split(' ')[0]
        for c in string.punctuation:
            family=family.replace(c,'').strip()
        families.append(family)
    return families

df_all['Family']=extract_surname(df_all['Name'])

In [ ]:
df_all['Family'].nunique()

In [ ]:
df_all[['Title','Survived','Family_size']].groupby('Title').mean()

###JHU THINK & PRACTICE

In [ ]:
print("Survival_rates_grouped_by_families_of_women_in_dataset:")
df_all.loc[(df_all['Sex']=='female')&(df_all['Family_size']>1)].groupby('Family')['Survived'].mean().plot.hist(alpha=0.4)

###JHU how to read this chart, 0 to 1 on x axes and 0 to 100 on y axes???
###JHU and whats the point doing this


In [ ]:
master_families=df_all[df_all['Title']=='Master']['Family'].tolist()
df_all.loc[df_all['Family'].isin(master_families)].groupby('Family')['Survived'].mean().plot.hist(alpha=0.4)

###JHU how to read this chart, 0 to 1 on x axes and 0 to 100 on y axes???


In [ ]:
#Create variables with the survival rate of the women and masters

women_rate=df_all.loc[(df_all['Sex']=='female')&(df_all['Family_size']>1)].groupby('Family')['Survived'].mean()
master_rate=df_all.loc[df_all['Family'].isin(master_families)].groupby('Family')['Survived'].mean()

#Combine two series
combined_rate=women_rate.append(master_rate)

#It is posible that a woman has the family as a master and vice versa, so duplicates have to been dropped
combined_rate_df=combined_rate.to_frame().reset_index().rename(columns={'Survived':'Survival_quota'}).drop_duplicates(subset='Family')

#Merge the new dataframe
df_all=pd.merge(df_all,combined_rate_df,how='left')

In [ ]:
#We have calculated a survival rate for only a part of the cases, the others 
#we set to 0 in the dummy variable

df_all['Survival_quota_NA']=1
df_all.loc[df_all['Survival_quota'].isnull(),'Survival_quota_NA']=0
df_all['Survival_quota']=df_all['Survival_quota'].fillna(0)


In [ ]:
####3.3 Label and One Hot Encoding


In [ ]:
###JHU
from sklearn import preprocessing

non_numeric_features=['Embarked','Sex','Title','Age','Fare','Deck']

for feature in non_numeric_features:
    df_all[feature]=preprocessing.LabelEncoder().fit_transform(df_all[feature])
    
cat_features=['Pclass','Sex','Embarked','Title','Deck','Family_size_bin','Age','Fare']
    
encoded_features=[]

for feature in cat_features:
    encoded_feat=preprocessing.OneHotEncoder().fit_transform(df_all[feature].values.reshape(-1,1)).toarray()
    n=df_all[feature].nunique()
    cols=['{}_{}'.format(feature,n) for n in range(1,n+1)]
    encoded_df=pd.DataFrame(encoded_feat,columns=cols)
    encoded_df.index=df_all.index
    encoded_features.append(encoded_df)


df_all=pd.concat([df_all,*encoded_features],axis=1)

In [ ]:
df_train,df_test=divide_df(df_all)

In [ ]:
####Modeling and prediction


In [ ]:
#Defined columns which can be dropped for the modeling part because we 
#created new lable and one hot encoded variants out of them

drop_cols=['Embarked','Family','Family_size','Survived','Family_size_bin',
           'Deck','Age','Name','Parch','PassengerId','Pclass','Sex','SibSp',
           'Title','Ticket','Cabin']

drop_cols_2=['Embarked','Family','Family_size','Family_size_bin','Deck','Fare',
            'Name','Parch','PassengerId','Pclass','Sex','SibSp','Title',
             'Ticket','Cabin']


In [ ]:
#setting up a random forest classifier
#standardization of variables

###JHU
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
###JHU from sklearn.preprocessing import StandardScaler

x=preprocessing.StandardScaler().fit_transform(df_train.drop(columns=drop_cols))
y=df_train['Survived'].values

x_test=preprocessing.StandardScaler().fit_transform(df_test.drop(columns=drop_cols_2))

#creating train, test splits
x_train,x_test1,y_train,y_test1=train_test_split(x,y,test_size=0.25,random_state=42)

#defining model parameters
model=RandomForestClassifier(criterion='gini',
                                          n_estimators=1750,
                                          max_depth=7,
                                          min_samples_split=6,
                                          min_samples_leaf=6,
                                          max_features=True,
                                          oob_score=True,
                                          random_state=42,
                                          n_jobs=-1,
                                          verbose=1)

model.fit(x_train,y_train)
predictions=model.predict(x_test)
print(model.score(x_test1,y_test1))
output=pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':predictions})
output['Survived']=output['Survived'].astype(int)
output.to_csv('2020_00_09_titanic_jjj.csv',index=False)